### 앙상블 사용법
클래스는 `ensemble.py`에서 확인  

**`config_paths`**  
fine-tuning 시 사용했던 config.yaml의 경로를 string list로 입력  
이때 모델과 config.yaml은 해당 위치의 `models`라는 폴더 안에 넣어두기  
config.yaml의 model_name에서 배포자명을 제외한 모델의 이름과 저장한 pt 파일의 이름이 일치하도록 하기(문제가 된다면 수정 예정)  
- 예:  
  ```yaml
  model_name: klue/roberta-small
  ...
  ```
  위와 같이 지정된 경우, 저장된 모델의 이름은 `roberta-small.pt`

**`base_predictions()`**  
fine-tuning을 수행한 LLM들을 사용해서 base predictions를 추출  
데이터 개수만큼의 행, 모델 수만큼의 열이 생성  

**`stacking`, `kfold_stacking`, `soft_voting`**  
각각 기본 stacking, soft voting, kfold stacking으로 메타 모델을 학습  
clf 인자에 'linear'를 입력하면 LinearRegression을, 'lgbm'을 입력하면 LightBGM 모델을 사용하고 클래스의 멤버 변수로 등록  
clf를 바꾸고 싶다면 clf 인자를 변경해서 함수를 다시 호출하면 됨  
kfold stacking은 원래 kfold를 적용한 데이터로 LLM부터 학습해야 하지만 시간 관계 상, 리소스 관계 상 메타 모델에만 k-fold를 적용하는 방식으로 구현  
n 값으로 몇 개의 fold를 사용할지 지정  

**`inference`**  
sample_submission.csv를 읽어와 앙상블 결과 저장  
is_voting이 True이면 soft voting 결과를 사용하고, False이면 stacking과 kfold_stacking 중 가장 마지막에 사용한 classifier를 기준으로 추론 수행  

**`simulate`**  
임의로 두 개의 문장을 생성하여 추론을 수행하면 라벨 값이 출력  
마찬가지로 is_voting이 True이면 voting의 결과를 사용하고, False이면 stacking과 kfold stacking 중 가장 마지막에 사용한 classifier를 기준으로 수행  
앙상블에 사용한 모델의 base prediction과 최종 앙상블 결과가 함께 반환  
이 때 base prediction은 config_paths에서 지정한 순서와 같은 순서로 저장됨  

In [8]:
from ensemble import Ensemble

# config_paths = ['./models/kf_deberta_cross_sts_config.yaml', './models/electra-kor-base.yaml'] #9362
config_paths = ['./models/kf_deberta_cross_sts_config.yaml', './models/KR-ELECTRA-discriminator.yaml']
ensemble = Ensemble(config_paths=config_paths,
                    train_path='../../train_preprocess_v1.csv',
                    valid_path='../../dev_preprocess_v1.csv',
                    test_path='../../test_preprocess_v1.csv')

print(ensemble.X_train_base.shape, ensemble.y_train.shape)
print(ensemble.X_valid_base.shape, ensemble.y_valid.shape)
print(ensemble.X_test_base.shape)

========== base predicting train data ==========
+++++ Right now using "deliciouscat/kf-deberta-base-cross-sts" +++++


base prediction: 100%|██████████| 583/583 [02:13<00:00,  4.37it/s]


+++++ Right now using "snunlp/KR-ELECTRA-discriminator" +++++


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
base prediction: 100%|██████████| 1166/1166 [01:31<00:00, 12.79it/s]




========== base predicting valid data ==========
+++++ Right now using "deliciouscat/kf-deberta-base-cross-sts" +++++


...



========== base predicting test data ==========
+++++ Right now using "deliciouscat/kf-deberta-base-cross-sts" +++++


base prediction: 100%|██████████| 1100/1100 [00:31<00:00, 35.19it/s]


+++++ Right now using "snunlp/KR-ELECTRA-discriminator" +++++


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
base prediction: 100%|██████████| 1100/1100 [00:16<00:00, 66.27it/s]



(9324, 2) (9324, 1)
(550, 2) (550, 1)
(1100, 2)


In [9]:
ensemble.stacking(clf="linear")
ensemble.stacking(clf="xgboost")
ensemble.soft_voting()
ensemble.kfold_stacking("linear", 3)
ensemble.kfold_stacking("xgboost", 3)

========== linear stacking result ==========
    train pearson sim: [0.99864953]
    valid pearson sim: [0.93002911]


========== xgboost stacking result ==========
    train pearson sim: [0.99845927]
    valid pearson sim: [0.9281275]


========== soft voting result ==========
    train pearson sim: [0.99503921]
    valid pearson sim: [0.93313693]


========== linear stacking result ==========
    train pearson sim: [0.99864944]
    valid pearson sim: [0.92997726]


========== xgboost stacking result ==========
    train pearson sim: [0.9982874]
    valid pearson sim: [0.92768993]




(array([0.9982874]), array([0.92768993]))

In [10]:
# ensemble.stacking("linear")
ensemble.inference(is_voting=True, submission_path='../../sample_submission.csv')

saved ensemble_output.csv


In [14]:
import numpy as np

sentence_1 = "손 틈 새로 비치는"
sentence_2 = "아이유 참 좋다"
voting_base, voting_ensemble = ensemble.simulate(sentence_1, sentence_2, True)
_, stacking_ensemble = ensemble.simulate(sentence_1, sentence_2, False)

print(f"base predictions: {np.round(voting_base, 1)}, voting prediction: {voting_ensemble}, stacking prediction: {stacking_ensemble}")

Now Using deliciouscat/kf-deberta-base-cross-sts
Now Using snunlp/KR-ELECTRA-discriminator


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Now Using deliciouscat/kf-deberta-base-cross-sts
Now Using snunlp/KR-ELECTRA-discriminator
base predictions: [[ 0.1 -0. ]], voting prediction: [0.], stacking prediction: [1.7]
